# Topic modeling para *Big Data*
Vamos a ver cómo realizar un modelado de temática en grandes volúmenes de texto con la librería `gensim`  

Utilizaremos el conjunto de datos *Lee* de `Gensim` (es una versión abreviada del conjunto http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF).  

Para visualizar gráficamente los tópicos es necesario instalar la librería `pyLDAvis` dentro del entorno de Anaconda con el comando:
```python
conda install -c conda-forge pyldavis 
```

### Cargamos librerías

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
import warnings

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
warnings.filterwarnings('ignore')

# spacy para lematizar
import spacy

# herramientas de dibujado
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

Utilizamos un generador para obtener los documentos del Corpus línea a línea desde el archivo del conjunto de ejemplo y convertirlos en un listado de tokens.

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
stop_words = nlp.Defaults.stop_words #listado de stop-words

def lemmatize_corpus(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """Función que devuelve el lema de una string,
    excluyendo las palabras cuyo POS_TAG no está en la lista"""
    text_out = [t.lemma_.lower() for t in nlp(text)
                if t.pos_ in allowed_postags
                and len(t.lemma_)>3
                and not t.is_stop]
    return text_out
            
def build_texts(fname):
    """
    Generador que devuelve el texto tokenizado a partir de un archivo
    línea a línea
    """
    with open(fname) as f:
        for line in f:
            yield lemmatize_corpus(line)

In [ ]:
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_data_file = data_dir + os.sep + 'lee_background.cor'

In [ ]:
lee_data_file

In [ ]:
texto=build_texts(lee_data_file)

In [ ]:
texto

In [ ]:
print(next(texto))

In [ ]:
for t in texto:
    print(t)
    break

### Creamos el diccionario y el corpus para Topic Modeling
Las dos entradas para el modelo LDA son un diccionario de `gensim` y un corpus de texto.  
Preparamos el diccionario:

In [ ]:
class BOW_Corpus(object):
    """
    Iterable: en cada iteración devuelve el vector bag-of-words
    del siguiente documento en el corpus.
    El corpus es el listado de críticas alojadas en el directorio
    pasado como argumento al instanciar la clase.
    
    Procesa un documento cada vez, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, filename):
        self.filename = filename
        #creamos bigramas y trigramas
        self.bigram = gensim.models.Phrases(build_texts(self.filename), min_count=5, threshold=50) # higher threshold fewer phrases.
        #optimizamos una vez entreando
        self.bigram_mod = gensim.models.phrases.Phraser(self.bigram)

        self.trigram = gensim.models.Phrases(self.bigram_mod[build_texts(self.filename)], min_count=5, threshold=50)  
        self.trigram_mod = gensim.models.phrases.Phraser(self.trigram)
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.diccionario = gensim.corpora.Dictionary(self.trigram_mod[map(lambda x: self.bigram_mod[x], build_texts(self.filename))])

    def __len__(self):
        #necesitamos saber la longitud del corpus para visualizar con pyLDAvis
        return self.diccionario.num_docs
    
    def __iter__(self):
        """
        __iter__ es un iterable => BOW_Corpus es un streamed iterable.
        """
        for tokens in build_texts(self.filename):
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.diccionario.doc2bow(self.trigram_mod[self.bigram_mod[tokens]])

In [ ]:
corpus_bow = BOW_Corpus(lee_data_file)

In [ ]:
len(corpus_bow)

In [ ]:
corpus_bow.diccionario.num_docs

In [ ]:
len(corpus_bow.diccionario.token2id)

In [ ]:
[k for k in corpus_bow.diccionario.token2id if re.match(r'\w+_\w+_\w+', k)]

In [ ]:
len([k for k in corpus_bow.diccionario.token2id if re.match(r'\w+_\w+', k)])

In [ ]:
for c in corpus_bow:
    print(c)
    print(len(c))
    break

### Modelo LDA
Es un modelo generativo que considera cada documento como una mezcla de temas donde cada tema tiene una distribución de las palabras.

In [ ]:
warnings.filterwarnings('ignore')


ldamodel = LdaModel(corpus=corpus_bow, num_topics=10, id2word=corpus_bow.diccionario)
pprint(ldamodel.print_topics())

### Visualización de los temas  
Podemos visualizarlo gráficamente la distribución de los documentos del Corpus por temas con la librería `pyLDAvis`

In [ ]:
warnings.filterwarnings('ignore')
vis_data = gensimvis.prepare(ldamodel, corpus_bow, corpus_bow.diccionario)
pyLDAvis.display(vis_data)

In [ ]:
for c in corpus_bow:
    print(c)
    break

In [ ]:
#reducimos el vocabulario
corpus_bow.diccionario.filter_extremes(no_above=0.7)

In [ ]:
len(corpus_bow.diccionario.token2id)

In [ ]:
for c in corpus_bow:
    print(len(c))
    break

In [ ]:
warnings.filterwarnings('ignore')
ldamodel = LdaModel(corpus=corpus_bow, num_topics=10, id2word=corpus_bow.diccionario)
vis_data = gensimvis.prepare(ldamodel, corpus_bow, corpus_bow.diccionario)
pyLDAvis.display(vis_data)

In [ ]:
for l in ldamodel[corpus_bow]:
    print(l)
    break

In [ ]:
ldamodel[corpus_bow]